In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd

In [2]:
def visit_check_url(url, browser_to_use = 'chrome', text_check = 'Mars'):
    #set up the browser to use
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser(browser_to_use, **executable_path, headless=False)
    #visit the given url
    browser.visit(url)
    #it returns a positive message if there is text in the url and a negative one if it is empty
    if browser.is_text_present(text_check):
        print("Yes, the official website was found!")
    else:
        print("No, it wasn't found...")
    return(browser)

# NASA Mars News

In [3]:
url_nasa = 'https://mars.nasa.gov/news/'
#check if you can find the website
browser = visit_check_url(url_nasa)
#creating a BEautifulSoup object
soup = bs(browser.html, 'html.parser')
#reading the latest News Title
news_title = soup.find_all('div', class_='content_title')[0].text
#reading the latest news body
paragraph = soup.find_all('div', class_="article_teaser_body")[0].text
#printing the title and body just to see them and check that they are correct
print(f'{news_title}\n{paragraph}')
    
browser.quit()

Yes, the official website was found!
Mars Now
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  


# JPL Mars Space Images

In [4]:
url_jpl = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
#check if you can find the website
browser = visit_check_url(url_jpl)
#clicking on the button FULL IMAGE to see the full image featured this month  class="btn btn-outline-light"
browser.links.find_by_partial_text('FULL IMAGE').click()
#scraping the page with BS
soup = bs(browser.html, 'html.parser')
#saving the image url, sometimes it does not find mars3.jpg I am not sure why
featured_image_url = url_jpl.\
    replace("index.html", soup.find(class_="fancybox-image")["src"]) 
print(featured_image_url)
browser.quit()

Yes, the official website was found!
https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


# Mars Facts

In [5]:
url_mars_facts = 'https://space-facts.com/mars/'
#check if you can find the website
browser = visit_check_url(url_mars_facts)
browser.quit()
# reading the table from the url using pandas
table = pd.read_html(url_mars_facts)
#selecting the table containing Mars information
df_facts = table[0]
 #renaming the columns
df_facts.columns = ['Description', 'Mars']
# setting the description of the parameters as index
df_facts = df_facts.set_index('Description')
#saving the table as an HTML and stripping \n character not known by html.
html_facts = df_facts.to_html(index_names=False, justify='center', classes="table table-striped table-sm", border = 0)
html_facts = html_facts.replace('\n','')
df_facts.head()

Yes, the official website was found!


,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [6]:
html_facts

'<table border="0" class="dataframe table table-striped table-sm">  <thead>    <tr style="text-align: center;">      <th></th>      <th>Mars</th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

# Mars Hemispheres

In [7]:
url_hemisphere = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser = visit_check_url(url_hemisphere)
html = browser.html
#check if you can find the website
soup = bs(html, 'html.parser')
#finding all the items and possible images
items = soup.find_all(class_='item')
#getting information on the images such as title and urls to find the full image
#titles = []
#images_urls = []
hemispheres = []
for item in items:
    item_dic = {}
    #finding the titles of the images
    title = item.find('h3').text
    item_dic['Title'] = title
    #clicking in every image to retrieve the full size picture
    browser.links.find_by_partial_text(title).click()
    html = browser.html
    soup = bs(html, 'html.parser')
    #finding the urls of the picture
    url_1 = soup.find('div', class_='downloads')
    url = url_1.find('a')['href']
    item_dic['img_url'] = url
    browser.back()
    hemispheres.append(item_dic)
    
browser.quit()

Yes, the official website was found!


In [8]:
for hemisphere in hemispheres:
    print(hemisphere)

{'Title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
{'Title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
{'Title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
{'Title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}
